In [1]:
import pandas as pd

In [52]:
# Ler e printar dados
Dados = pd.read_csv("Data\Base_Completa.csv", sep=";", index_col="Horario")
Dados.head()

,Tweets,Sentimento,Close,Retornos
Horario,,,,
2022-03-02 03:00:00+00:00,RT @WatcherGuru: BREAKING: Airbnb CEO says the...,0.5994,44370.246094,NaN
2022-03-02 03:00:00+00:00,RT @nima33514465: @BluesparrowC @TheMoonCarl @...,0.3612,44370.246094,NaN
2022-03-02 03:00:00+00:00,RT @ICOAnnouncement: 🎙 New Featured Project\r\...,0.5859,44370.246094,NaN
2022-03-02 03:00:00+00:00,RT @AirdropStario: 💧 Farcana Airdrop 💧\r\n\r\n...,0.4753,44370.246094,NaN
2022-03-02 03:00:00+00:00,RT @AirdropStario: 💧 Farcana Airdrop 💧\r\n\r\n...,0.4753,44370.246094,NaN


In [54]:
# Transformar o índice em objeto de datas (DateTime)
Dados.index = pd.to_datetime(Dados.index)
Dados.index

DatetimeIndex(['2022-03-02 03:00:00+00:00', '2022-03-02 03:00:00+00:00',
               '2022-03-02 03:00:00+00:00', '2022-03-02 03:00:00+00:00',
               '2022-03-02 03:00:00+00:00', '2022-03-02 03:00:00+00:00',
               '2022-03-02 03:00:00+00:00', '2022-03-02 03:00:00+00:00',
               '2022-03-02 03:00:00+00:00', '2022-03-02 03:00:00+00:00',
               ...
               '2022-04-06 02:00:00+00:00', '2022-04-06 02:00:00+00:00',
               '2022-04-06 02:00:00+00:00', '2022-04-06 02:00:00+00:00',
               '2022-04-06 02:00:00+00:00', '2022-04-06 02:00:00+00:00',
               '2022-04-06 02:00:00+00:00', '2022-04-06 02:00:00+00:00',
               '2022-04-06 02:00:00+00:00', '2022-04-06 02:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Horario', length=93178, freq=None)

In [56]:
# Existem dados faltantes?
Dados.isna().sum()

# Os 100 valores NA da coluna retornos são causados pelo cálculo da vairação percentual, que não podem ser feitos para
# O primeiro ponto de dados, no caso, a primeira hora em que os tweets foram coletados.

Tweets          0
Sentimento      0
Close           0
Retornos      100
dtype: int64

In [57]:
# Dropar as linhas com valores nulos
Dados = Dados.dropna()
Dados.isna().sum()

Tweets        0
Sentimento    0
Close         0
Retornos      0
dtype: int64

In [59]:
# Conferir os valores da coluna retorno
Dados.head()

,Tweets,Sentimento,Close,Retornos
Horario,,,,
2022-03-02 04:00:00+00:00,"RT @BitcoinStimulus: $7,852 (+554%)\r\n\r\n#bi...",0.0000,44273.210938,-0.002187
2022-03-02 04:00:00+00:00,@TheMoonCarl You say you’re willing to gain we...,0.5994,44273.210938,-0.002187
2022-03-02 04:00:00+00:00,RT @Cointelegraph: Some on-chain stats still b...,0.0000,44273.210938,-0.002187
2022-03-02 04:00:00+00:00,"RT @bre_jew: @bitcoin_brown omg, I'm getting f...",-0.5574,44273.210938,-0.002187
2022-03-02 04:00:00+00:00,RT @BTC_Archive: 💥El Salvador President: The i...,0.3400,44273.210938,-0.002187


In [60]:
# Observar correlações entre as variáveis numéricas
Dados.corr()

# Correlação dos retornos com o valor do sentimento é extremamente pequena

,Sentimento,Close,Retornos
Sentimento,1.000000,-0.018273,-0.001543
Close,-0.018273,1.000000,0.028697
Retornos,-0.001543,0.028697,1.000000


In [28]:
# Exemplo de pegar sentimento para uma hora específica
Dados.loc["2022-03-02 03"]["Sentimento"]

Horario
2022-03-02 03:00:00+00:00    0.5994
2022-03-02 03:00:00+00:00    0.3612
2022-03-02 03:00:00+00:00    0.5859
2022-03-02 03:00:00+00:00    0.4753
2022-03-02 03:00:00+00:00    0.4753
                              ...  
2022-03-02 03:00:00+00:00    0.0000
2022-03-02 03:00:00+00:00    0.6124
2022-03-02 03:00:00+00:00    0.8013
2022-03-02 03:00:00+00:00    0.0000
2022-03-02 03:00:00+00:00    0.4753
Name: Sentimento, Length: 100, dtype: float64

In [61]:
# Somar os valores de sentimento para uma hora, a fim de se obter o sentimento geral.
# Se o total >0, então o sentimento naquela hora foi positivo, se não, foi negativo

def Obter_Sentimento_Agregado(mes,dia,hora):
    hora = f"0{hora}" if hora <10 else hora
    Agreg_numerico = Dados.loc[f"2022-0{mes}-0{dia} {hora}"]["Sentimento"].sum()
    Sentimento = "Positivo" if Agreg_numerico > 0 else "Negativo"
    return Sentimento

# Resultados
Obter_Sentimento_Agregado(3, 4, 4)

'Positivo'